In [ ]:
#!curl -L https://d36yt3mvayqw5m.cloudfront.net/public/nuimages-v1.0/nuimages-v1.0-all-metadata.tgz > data_annotation.tgz # real annotation

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  609M  100  609M    0     0   100M      0  0:00:06  0:00:06 --:--:--  107M


In [ ]:
#!curl -L https://d36yt3mvayqw5m.cloudfront.net/public/nuimages-v1.0/nuimages-v1.0-all-samples.tgz > data_image.tgz # real images

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.2G  100 15.2G    0     0  14.4M      0  0:17:57  0:17:57 --:--:-- 12.4M


In [ ]:
# minified version just for testing
#!curl -L https://d36yt3mvayqw5m.cloudfront.net/public/nuimages-v1.0/nuimages-v1.0-mini.tgz > mini.tgz && tar -xzf mini.tgz && mv v1.0-mini v1.0-train

In [ ]:
from google.colab import drive # the real images data files take too long (about 17 minutes) to be downloaded, so we just mount it
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!tar -xzf /content/drive/MyDrive/nuimages-v1.0-all-metadata.tgz && tar -xzf /content/drive/MyDrive/nuimages-v1.0-all-samples.tgz # extract. this will take about 7 minutes

In [54]:
drive.flush_and_unmount() # unmount when we don't need it anymore, so we have more disk space

In [ ]:
!mkdir train test val && mkdir train/image test/image val/image train/label test/label val/label # for the yolo format

In [ ]:
import pandas as pd

def bbox_to_yolo(xmin, ymin, xmax, ymax, image_width, image_height):
    box_width = xmax - xmin
    box_height = ymax - ymin

    x_center = xmin + box_width / 2
    y_center = ymin + box_height / 2

    # Normalize by image size
    x_center_norm = x_center / image_width
    y_center_norm = y_center / image_height
    width_norm = box_width / image_width
    height_norm = box_height / image_height

    return x_center_norm, y_center_norm, width_norm, height_norm


def read_and_join(path_prefix):
    obj_ann = pd.read_json(path_prefix + 'object_ann.json')
    category = pd.read_json(path_prefix + 'category.json').rename(columns={"name": "category_name"})
    category['category_id'] = range(len(category))
    sample_data = pd.read_json(path_prefix + 'sample_data.json')
    # attribute = pd.read_json(path_prefix + 'attribute.json')

    # object annotation left join sample_data
    merged = obj_ann.merge(
        sample_data,
        how = 'left',
        right_on='token',
        left_on='sample_data_token',
        suffixes=('_left', '_right')
    )

    merged = merged.merge(
        category,
        how = 'left',
        left_on='category_token',
        right_on='token',
        suffixes=('_from_merged', '_from_category')
    )

    print (len(obj_ann))
    print (len(sample_data))
    print (len(category))

    return merged

# create list of train/val/test image file name list in a txt file (train/val.txt), so we can use cp/rsync with it
def create_image_filename_list(path_prefix, data_split, merged):
    if data_split not in ['train', 'val', 'test']:
        return -1

    df = merged[['filename']]
    filenames = set([])
    for index, row in df.iterrows():
        filenames.add(row['filename'])
    for fn in filenames:
        with open(data_split+".txt", 'a') as f:
            f.write(fn + "\n")

# create yolo label txt
def convert_to_yolo_format(path_prefix, data_split, merged):
    if data_split not in ['train', 'val', 'test']:
        return -1
    df = merged[['bbox', 'category_id', 'category_name', 'filename', 'width',  'height']]
    for index, row in df.iterrows():
        filename = row['filename'].split("/")[-1] # filename column is in the format of samples/CAM_BACK/n010-2018-08-27-16-15-24+0800...
        abs_path_to_label_txt = path_prefix + data_split + "/label/" + filename + '.txt' # example: /content/train/label/asdf.jpg.txt
        x_center, y_center, width, height = bbox_to_yolo(row['bbox'][0], row['bbox'][1], row['bbox'][2], row['bbox'][3], row['width'], row['height'])
        with open(abs_path_to_label_txt, 'a') as f: # append label to the txt
            f.write(str(row['category_id']) + " " + str(x_center) + " " + str(y_center) + " " + str(width) + " " + str(height) + "\n")

path_prefix = '/content/v1.0-val.'
val_df = read_and_join('/content/v1.0-val/')
train_df = read_and_join('/content/v1.0-train/')

136074
213185
25
557715
872181
25


In [ ]:
create_image_filename_list('/content/', 'train', train_df)
create_image_filename_list('/content/', 'val', val_df)

In [47]:
!rm -rf /content/test/image/* /content/val/image/*

In [48]:
# copy the val image to directory expected by yolo
!xargs -a val.txt -P8 -I{} cp "/content/{}" /content/val/image/

In [ ]:
# NOTE: rsync alternative. it does the same thing with the cp
# # not only the image, this rsync will copy the (parent) directory structure as well (the samples/CAM_BACK) which is what we don't want
# !rsync -av --files-from=val.txt /content/ /content/temp_val_images/
# # so we do rsync once again, from the temp_val_images to val/image, which is the final directory
# !rsync -av /content/temp_val_images/samples/CAM_BACK/ /content/temp_val_images/samples/CAM_BACK_LEFT/ /content/temp_val_images/samples/CAM_BACK_RIGHT/ /content/temp_val_images/samples/CAM_FRONT/ /content/temp_val_images/samples/CAM_FRONT_LEFT/ /content/temp_val_images/samples/CAM_FRONT_RIGHT/ /content/val/image/
# !rm -rf /content/temp_val_images/ # remove, dont need anymore

building file list ... done
samples/
samples/CAM_BACK/
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536696137005.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536699437006.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536704887005.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536716387005.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536719387005.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536722137005.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536743137005.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536747687005.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536750687005.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536753287005.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536756887005.jpg
samples/CAM_BACK/n013-2018-08-06-14-15-50+0800__CAM_BACK__1533536764537005.j

In [49]:
# copy the train image to directory expected by yolo
!xargs -a train.txt -P8 -I{} cp "/content/{}" /content/train/image/

In [52]:
# NOTE: rsync alternative. it does the same thing with the cp
# # not only the image, this rsync will copy the (parent) directory structure as well (the samples/CAM_BACK) which is what we don't want
# !rsync -av --files-from=train.txt /content/ /content/temp_train_images/
# # so we do rsync once again, from the temp_val_images to val/image, which is the final directory
# !rsync -av /content/temp_train_images/samples/CAM_BACK/ /content/temp_train_images/samples/CAM_BACK_LEFT/ /content/temp_train_images/samples/CAM_BACK_RIGHT/ /content/temp_train_images/samples/CAM_FRONT/ /content/temp_train_images/samples/CAM_FRONT_LEFT/ /content/temp_train_images/samples/CAM_FRONT_RIGHT/ /content/train/image/
# !rm -rf /content/temp_train_images/ # remove, dont need anymore

In [53]:
train_df['filename'].head()

,filename
0,samples/CAM_BACK/n010-2018-08-27-16-15-24+0800...
1,samples/CAM_BACK_LEFT/n009-2018-09-12-09-59-51...
2,samples/CAM_BACK/n003-2018-01-08-11-30-34+0800...
3,samples/CAM_FRONT_RIGHT/n008-2018-05-24-12-02-...
4,samples/CAM_FRONT_RIGHT/n014-2018-06-20-21-35-...


In [ ]:
convert_to_yolo_format('/content/', 'train', train_df)

In [ ]:
convert_to_yolo_format('/content/', 'val', val_df)

In [50]:
!ls -1 /content/train/image/ | wc -l

60668


In [51]:
!ls -1 /content/train/label/ | wc -l

60668
